In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent  # Notebooks are in notebooks/, go up one level
sys.path.insert(0, str(project_root))

# Add venv packages
venv_path = project_root / "venv/lib/python3.13/site-packages"
if venv_path.exists():
    sys.path.insert(0, str(venv_path))

import numpy as np
import polars as pl
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib as mpl

# Set matplotlib backend for notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['figure.figsize'] = (12, 6)

print(f"✅ Project root: {project_root}")
print(f"✅ Src exists: {(project_root / 'src').exists()}")
print(f"✅ Matplotlib version: {mpl.__version__}")

✅ Project root: /Users/sudip/hull_tactical_market_prediction_using_hyperopt
✅ Src exists: True


In [8]:
# --- Import your modules ---
from src.features import create_advanced_features, select_features
from src.models.elastic_net import ElasticNetModel
from src.models.lightgbm_model import LightGBMModel
from src.models.xgboost_model import XGBoostModel
from src.models.ensemble import EnsembleModel
from evaluation import score

/Users/sudip/hull_tactical_market_prediction_using_hyperopt/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# --- Visualize Results ---

def plot_model_predictions(signals_dict):
    """Plot predictions from all models."""
    fig, axes = plt.subplots(len(signals_dict), 1, figsize=(14, 4*len(signals_dict)))
    
    if len(signals_dict) == 1:
        axes = [axes]
    
    for i, (name, signals) in enumerate(signals_dict.items()):
        axes[i].plot(signals, marker='o', linewidth=2, label=name)
        axes[i].set_title(f'{name} Predictions', fontsize=14, fontweight='bold')
        axes[i].set_xlabel('Test Sample Index')
        axes[i].set_ylabel('Signal Value')
        axes[i].axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Neutral')
        axes[i].axhspan(0, 2, alpha=0.1, color='green', label='Valid Range')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('artifacts/predictions_comparison.png', dpi=300, bbox_inches='tight')
    print("✅ Saved predictions plot to artifacts/predictions_comparison.png")
    plt.show()

def plot_feature_importance(feature_importance, top_n=20):
    """Plot feature importance."""
    sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:top_n]
    features, importances = zip(*sorted_features)
    
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(features)), importances, color='steelblue')
    plt.yticks(range(len(features)), features)
    plt.xlabel('Importance Score', fontsize=12)
    plt.title(f'Top {top_n} Feature Importance (LightGBM)', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    
    # Ensure artifacts directory exists
    Path('artifacts').mkdir(exist_ok=True)
    plt.savefig('artifacts/feature_importance.png', dpi=300, bbox_inches='tight')
    print(f"✅ Saved feature importance plot to artifacts/feature_importance.png")
    plt.show()

def plot_model_comparison(signals_dict):
    """Compare models side by side."""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Predictions comparison
    for name, signals in signals_dict.items():
        axes[0].plot(signals, marker='o', label=name, linewidth=2, markersize=8)
    axes[0].set_title('Model Predictions Comparison', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Test Sample Index')
    axes[0].set_ylabel('Signal Value')
    axes[0].axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Neutral')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Signal distribution
    for name, signals in signals_dict.items():
        axes[1].hist(signals, alpha=0.6, label=name, bins=20)
    axes[1].set_title('Signal Distribution', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Signal Value')
    axes[1].set_ylabel('Frequency')
    axes[1].axvline(x=1.0, color='r', linestyle='--', alpha=0.5, label='Neutral')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    Path('artifacts').mkdir(exist_ok=True)
    plt.savefig('artifacts/model_comparison.png', dpi=300, bbox_inches='tight')
    print("✅ Saved comparison plot to artifacts/model_comparison.png")
    plt.show()


In [ ]:
# --- Visualize Training Results ---

# Plot model predictions
plot_model_predictions(all_signals)

# Plot model comparison
plot_model_comparison(all_signals)

# Plot feature importance if available
try:
    lgbm_model = results['LightGBM'][0]
    feature_importance = lgbm_model.get_feature_importance()
    plot_feature_importance(feature_importance, top_n=20)
except Exception as e:
    print(f"Could not plot feature importance: {e}")


In [9]:
def load_data(data_path: Path):
    """Load and preprocess data."""
    print("Loading data...")
    
    train = (
        pl.read_csv(data_path / "train.csv")
        .rename({'market_forward_excess_returns': 'target'})
        .with_columns(pl.exclude('date_id').cast(pl.Float64, strict=False))
        .head(-10)
    )
    
    test = (
        pl.read_csv(data_path / "test.csv")
        .rename({'lagged_forward_returns': 'target'})
        .with_columns(pl.exclude('date_id').cast(pl.Float64, strict=False))
    )
    
    print(f"Train shape: {train.shape}, Test shape: {test.shape}")
    return train, test

In [10]:
def create_features(df: pl.DataFrame) -> pl.DataFrame:
    """Create features from raw data."""
    print("Creating features...")
    
    vars_to_keep = ["S2", "E2", "E3", "P9", "S1", "S5", "I2", "P8", "P10", "P12", "P13"]
    
    # Create basic features
    df = df.with_columns([
        (pl.col("I2") - pl.col("I1")).alias("U1") if all(c in df.columns for c in ["I2", "I1"]) else pl.lit(None),
        (pl.col("M11") / ((pl.col("I2") + pl.col("I9") + pl.col("I7")) / 3)).alias("U2") if all(c in df.columns for c in ["M11", "I2", "I9", "I7"]) else pl.lit(None)
    ])
    
    df = df.select(["date_id", "target"] + vars_to_keep + ["U1", "U2"])
    
    # Fill nulls
    feature_cols = [col for col in df.columns if col not in ['date_id', 'target']]
    df = df.with_columns([
        pl.col(col).fill_null(pl.col(col).ewm_mean(com=0.5))
        for col in feature_cols
    ])
    
    # Create advanced features
    df = create_advanced_features(df, feature_cols)
    
    return df.drop_nulls()

In [11]:
def prepare_data(train: pl.DataFrame, test: pl.DataFrame):
    """Prepare train and test datasets."""
    print("Preparing data...")
    
    # Join and create features
    common_columns = [col for col in train.columns if col in test.columns]
    df = pl.concat([train.select(common_columns), test.select(common_columns)], how="vertical")
    
    df = create_features(df)
    
    # Split back
    train_date_ids = set(train.get_column('date_id'))
    test_date_ids = set(test.get_column('date_id'))
    
    train_df = df.filter(pl.col('date_id').is_in(list(train_date_ids)))
    test_df = df.filter(pl.col('date_id').is_in(list(test_date_ids)))
    
    # Get features
    feature_cols = select_features(train_df, method='all')
    print(f"Number of features: {len(feature_cols)}")
    
    # Prepare X and y
    X_train = train_df.select(feature_cols).to_numpy()
    y_train = train_df.get_column('target').to_numpy()
    
    X_test = test_df.select(feature_cols).to_numpy()
    y_test = test_df.get_column('target').to_numpy()
    
    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, y_train, X_test, y_test, scaler, feature_cols, test_df

In [12]:
def evaluate_model(name: str, predictions: np.ndarray, test_data: pl.DataFrame):
    """Evaluate model performance."""
    print(f"\nEvaluating {name}...")
    
    # Clip predictions to [0, 2] range
    signals = np.clip(predictions * 400 + 1, 0.0, 2.0)
    
    print(f"Signal range: [{signals.min():.4f}, {signals.max():.4f}]")
    print(f"Signal mean: {signals.mean():.4f}")
    
    return signals

In [13]:
def train_all_models(X_train, y_train, X_test, optimize=True):
    """Train all models and return predictions."""
    results = {}
    
    # ElasticNet
    print("\n" + "="*50)
    print("Training ElasticNet...")
    print("="*50)
    enet = ElasticNetModel(n_trials=20 if optimize else 0)
    enet.fit(X_train, y_train, optimize=optimize)
    results['ElasticNet'] = (enet, enet.predict(X_test))
    
    # LightGBM
    print("\n" + "="*50)
    print("Training LightGBM...")
    print("="*50)
    lgbm = LightGBMModel(n_trials=20 if optimize else 0)
    lgbm.fit(X_train, y_train, optimize=optimize)
    results['LightGBM'] = (lgbm, lgbm.predict(X_test))
    
    # XGBoost
    print("\n" + "="*50)
    print("Training XGBoost...")
    print("="*50)
    xgb_model = XGBoostModel(n_trials=20 if optimize else 0)
    xgb_model.fit(X_train, y_train, optimize=optimize)
    results['XGBoost'] = (xgb_model, xgb_model.predict(X_test))
    
    # Ensemble (simple average)
    print("\n" + "="*50)
    print("Training Ensemble...")
    print("="*50)
    ensemble_models = [enet, lgbm, xgb_model]
    ensemble = EnsembleModel(ensemble_models, weights=[0.3, 0.35, 0.35])
    ensemble.fit(X_train, y_train, optimize=False)
    results['Ensemble'] = (ensemble, ensemble.predict(X_test))
    
    return results

In [14]:
def main():
    """Main training function."""
    print("="*70)
    print("Hull Tactical Market Prediction - Advanced Training")
    print("="*70)
    
    # Paths
    data_path = Path('/Users/sudip/hull_tactical_market_prediction_using_hyperopt/input/hull-tactical-market-prediction/')
    
    # Load data
    train, test = load_data(data_path)
    
    # Prepare data
    X_train, y_train, X_test, y_test, scaler, feature_cols, test_df = prepare_data(train, test)
    
    print(f"\nData shapes:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
    
    # Train models
    results = train_all_models(X_train, y_train, X_test, optimize=True)
    
    # Evaluate models
    print("\n" + "="*70)
    print("Model Evaluation Results")
    print("="*70)
    
    all_signals = {}
    for name, (model, predictions) in results.items():
        signals = evaluate_model(name, predictions, test_df)
        all_signals[name] = signals
    
    # Feature importance
    print("\n" + "="*70)
    print("Feature Importance (LightGBM)")
    print("="*70)
    try:
        lgbm_model = results['LightGBM'][0]
        feature_importance = lgbm_model.get_feature_importance()
        # Sort by importance
        sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
        for i, (feature, importance) in enumerate(sorted_features[:10]):
            print(f"{i+1}. {feature}: {importance:.4f}")
    except Exception as e:
        print(f"Could not get feature importance: {e}")
    
    return results, all_signals, test_df

In [15]:
if __name__ == "__main__":
    results, signals, test_data = main()
    print("\nTraining completed successfully!")

[I 2025-10-27 19:30:23,846] A new study created in memory with name: no-name-21c5b6ee-f00f-4834-b846-17bb9535d120


Hull Tactical Market Prediction - Advanced Training
Loading data...
Train shape: (8980, 98), Test shape: (10, 99)
Preparing data...
Creating features...
Number of features: 88

Data shapes:
X_train: (7469, 88), y_train: (7469,)
X_test: (10, 88), y_test: (10,)

Training ElasticNet...


Best trial: 0. Best value: 0.0109217:  10%|█         | 2/20 [00:00<00:00, 44.20it/s]

[I 2025-10-27 19:30:23,873] Trial 0 finished with value: 0.01092168130280988 and parameters: {'alpha': 0.007297800465703231, 'l1_ratio': 0.8266977037498363}. Best is trial 0 with value: 0.01092168130280988.
[I 2025-10-27 19:30:23,885] Trial 1 finished with value: 0.01092168130280988 and parameters: {'alpha': 0.01909701656454613, 'l1_ratio': 0.8129398095122538}. Best is trial 0 with value: 0.01092168130280988.
[I 2025-10-27 19:30:23,899] Trial 2 finished with value: 0.01092168130280988 and parameters: {'alpha': 0.2077975938957827, 'l1_ratio': 0.22910052531465752}. Best is trial 0 with value: 0.01092168130280988.


Best trial: 5. Best value: 0.0109063:  50%|█████     | 10/20 [00:00<00:00, 66.67it/s]

...

[I 2025-10-27 19:30:55,201] Trial 16 finished with value: 0.010141831568034786 and parameters: {'max_depth': 8, 'learning_rate': 0.09998781972141131, 'subsample': 0.5989009707694832, 'colsample_bytree': 0.9853167602144361, 'colsample_bylevel': 0.5821798281555711, 'min_child_weight': 7, 'gamma': 6.867214521653489e-05, 'reg_alpha': 7.92601621034049e-05, 'reg_lambda': 0.10562830430862857}. Best is trial 13 with value: 0.010033399530710621.
[I 2025-10-27 19:30:55,300] Trial 17 finished with value: 0.010922912023579038 and parameters: {'max_depth': 9, 'learning_rate': 0.10599230316702539, 'subsample': 0.5549192692289029, 'colsample_bytree': 0.5733565867739245, 'colsample_bylevel': 0.613878929285436, 'min_child_weight': 10, 'gamma': 9.727543893814603, 'reg_alpha': 0.00046731217430960186, 'reg_lambda': 0.1459772174162853}. Best is trial 13 with value: 0.010033399530710621.


Best trial: 13. Best value: 0.0100334:  95%|█████████▌| 19/20 [00:07<00:00,  2.03it/s]

[I 2025-10-27 19:30:55,880] Trial 18 finished with value: 0.010704892406837224 and parameters: {'max_depth': 8, 'learning_rate': 0.20812329098725613, 'subsample': 0.5806216097133163, 'colsample_bytree': 0.7833831791388495, 'colsample_bylevel': 0.5009588942914032, 'min_child_weight': 7, 'gamma': 3.600221195586818e-07, 'reg_alpha': 0.0003739291287389318, 'reg_lambda': 4.576183682981424}. Best is trial 13 with value: 0.010033399530710621.


Best trial: 19. Best value: 0.00971856: 100%|██████████| 20/20 [00:09<00:00,  2.09it/s]


[I 2025-10-27 19:30:57,974] Trial 19 finished with value: 0.009718563234343355 and parameters: {'max_depth': 10, 'learning_rate': 0.043968093469534064, 'subsample': 0.74238983461463, 'colsample_bytree': 0.8969790923646425, 'colsample_bylevel': 0.6343922657783546, 'min_child_weight': 7, 'gamma': 7.061409496890157e-05, 'reg_alpha': 0.2530963120406954, 'reg_lambda': 0.05297717476033028}. Best is trial 19 with value: 0.009718563234343355.
Best XGBoost score: 0.009719
Best XGBoost params: {'max_depth': 10, 'learning_rate': 0.043968093469534064, 'subsample': 0.74238983461463, 'colsample_bytree': 0.8969790923646425, 'colsample_bylevel': 0.6343922657783546, 'min_child_weight': 7, 'gamma': 7.061409496890157e-05, 'reg_alpha': 0.2530963120406954, 'reg_lambda': 0.05297717476033028}

Training Ensemble...

Model Evaluation Results

Evaluating ElasticNet...
Signal range: [0.8346, 1.1469]
Signal mean: 0.9746

Evaluating LightGBM...
Signal range: [0.0000, 2.0000]
Signal mean: 1.0527

Evaluating XGBoost